In [1]:
import numpy as np
import pandas as pd
#pd.set_option('max_columns', None)
#pd.set_option('max_rows', 90)

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')

from sklearn.neighbors import KNeighborsRegressor
import scipy.stats
from sklearn.preprocessing import StandardScaler
from pycaret.regression import setup, compare_models
from sklearn.model_selection import KFold, cross_val_score

from catboost import CatBoostRegressor
from sklearn.linear_model import BayesianRidge, HuberRegressor, Ridge, OrthogonalMatchingPursuit
from lightgbm import LGBMRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor

import optuna

In [2]:
titanic_train = pd.read_csv('C:\\Users\\Auditor\\Desktop\\Victor\\New folder\\Data Science\\Projects\\Titanic\\train.csv')

In [3]:
titanic_test = pd.read_csv('C:\\Users\\Auditor\\Desktop\\Victor\\New folder\\Data Science\\Projects\\Titanic\\test.csv')

In [4]:
titanic_train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [5]:
titanic_test

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [6]:
titanic_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.1+ KB


In [76]:
target = titanic_train['Survived']
test_ids = titanic_test['PassengerId']

In [77]:
titanic_train1 = titanic_train.drop(['PassengerId', 'Ticket','Survived','Name'], axis=1)
titanic_test1 = titanic_test.drop(['PassengerId','Ticket','Name'], axis=1)

In [78]:
titanic_train1

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,3,male,22.0,1,0,7.2500,NaN,S
1,1,female,38.0,1,0,71.2833,C85,C
2,3,female,26.0,0,0,7.9250,NaN,S
3,1,female,35.0,1,0,53.1000,C123,S
4,3,male,35.0,0,0,8.0500,NaN,S
...,...,...,...,...,...,...,...,...
886,2,male,27.0,0,0,13.0000,NaN,S
887,1,female,19.0,0,0,30.0000,B42,S
888,3,female,NaN,1,2,23.4500,NaN,S
889,1,male,26.0,0,0,30.0000,C148,C


In [79]:
titanic_test1

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,3,male,34.5,0,0,7.8292,NaN,Q
1,3,female,47.0,1,0,7.0000,NaN,S
2,2,male,62.0,0,0,9.6875,NaN,Q
3,3,male,27.0,0,0,8.6625,NaN,S
4,3,female,22.0,1,1,12.2875,NaN,S
...,...,...,...,...,...,...,...,...
413,3,male,NaN,0,0,8.0500,NaN,S
414,1,female,39.0,0,0,108.9000,C105,C
415,3,male,38.5,0,0,7.2500,NaN,S
416,3,male,NaN,0,0,8.0500,NaN,S


In [80]:
data1 = pd.concat([titanic_train1, titanic_test1], axis=0).reset_index(drop=True)
data1

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,3,male,22.0,1,0,7.2500,NaN,S
1,1,female,38.0,1,0,71.2833,C85,C
2,3,female,26.0,0,0,7.9250,NaN,S
3,1,female,35.0,1,0,53.1000,C123,S
4,3,male,35.0,0,0,8.0500,NaN,S
...,...,...,...,...,...,...,...,...
1304,3,male,NaN,0,0,8.0500,NaN,S
1305,1,female,39.0,0,0,108.9000,C105,C
1306,3,male,38.5,0,0,7.2500,NaN,S
1307,3,male,NaN,0,0,8.0500,NaN,S


In [81]:
target

0      0
1      1
2      1
3      1
4      0
      ..
886    0
887    1
888    0
889    1
890    0
Name: Survived, Length: 891, dtype: int64

In [82]:
data2 = data1.copy()

In [83]:
data2

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,3,male,22.0,1,0,7.2500,NaN,S
1,1,female,38.0,1,0,71.2833,C85,C
2,3,female,26.0,0,0,7.9250,NaN,S
3,1,female,35.0,1,0,53.1000,C123,S
4,3,male,35.0,0,0,8.0500,NaN,S
...,...,...,...,...,...,...,...,...
1304,3,male,NaN,0,0,8.0500,NaN,S
1305,1,female,39.0,0,0,108.9000,C105,C
1306,3,male,38.5,0,0,7.2500,NaN,S
1307,3,male,NaN,0,0,8.0500,NaN,S


In [84]:
data2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    1309 non-null   int64  
 1   Sex       1309 non-null   object 
 2   Age       1046 non-null   float64
 3   SibSp     1309 non-null   int64  
 4   Parch     1309 non-null   int64  
 5   Fare      1308 non-null   float64
 6   Cabin     295 non-null    object 
 7   Embarked  1307 non-null   object 
dtypes: float64(2), int64(3), object(3)
memory usage: 81.9+ KB


In [85]:
from sklearn.compose import ColumnTransformer
from sklearn.base import TransformerMixin
from sklearn.base import BaseEstimator
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

#Changing all null numeric values to mean
class MultipleColumnImputerMean(BaseEstimator, TransformerMixin):
    
    def __init__(self, columns):
        self.columns = columns
    
    def fit(self, X, y=None):
        self.imputer = SimpleImputer(strategy='mean')
        self.imputer.fit(X[self.columns])
        return self
    
    def transform(self, X):
        X_copy = X.copy()
        X_copy[self.columns] = self.imputer.transform(X_copy[self.columns])
        return X_copy

In [86]:
#Changing all null string values to mode

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer

class MultipleColumnImputerMode(BaseEstimator, TransformerMixin):
    
    def __init__(self, columns):
        self.columns = columns
    
    def fit(self, X, y=None):
        self.imputer = SimpleImputer(strategy='most_frequent')
        self.imputer.fit(X[self.columns])
        return self
    
    def transform(self, X):
        X_copy = X.copy()
        X_copy[self.columns] = self.imputer.transform(X_copy[self.columns])
        return X_copy

In [87]:
#gets dummies/ one_hot_encoder
class GetDummies(BaseEstimator, TransformerMixin):
    
    def __init__(self, column_names):
        self.column_names = column_names

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        columns = X[self.column_names]  # Extract the specified columns

        # Apply one-hot encoding to each specified column
        dummies_list = [pd.get_dummies(columns[col], prefix=col) for col in self.column_names]

        # Concatenate one-hot encoded columns with the original DataFrame and delete dummied column
        X_encoded = pd.concat([X] + dummies_list, axis=1).drop(self.column_names, axis=1)

        return X_encoded

In [88]:
from sklearn.pipeline import Pipeline
pipeline = Pipeline([
    ('MultipleColumnImputerMean', MultipleColumnImputerMean(columns=['Age', 'Fare'])),
    ('MultipleColumnImputerMode', MultipleColumnImputerMode(columns=['Cabin'])),
    ('getdummies', GetDummies(column_names=['Sex','Embarked']))])
    

In [89]:
data2

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,3,male,22.0,1,0,7.2500,NaN,S
1,1,female,38.0,1,0,71.2833,C85,C
2,3,female,26.0,0,0,7.9250,NaN,S
3,1,female,35.0,1,0,53.1000,C123,S
4,3,male,35.0,0,0,8.0500,NaN,S
...,...,...,...,...,...,...,...,...
1304,3,male,NaN,0,0,8.0500,NaN,S
1305,1,female,39.0,0,0,108.9000,C105,C
1306,3,male,38.5,0,0,7.2500,NaN,S
1307,3,male,NaN,0,0,8.0500,NaN,S


In [90]:
data3 = pipeline.fit_transform(data2)

In [91]:
data3

,Pclass,Age,SibSp,Parch,Fare,Cabin,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,3,22.000000,1,0,7.2500,C23 C25 C27,0,1,0,0,1
1,1,38.000000,1,0,71.2833,C85,1,0,1,0,0
2,3,26.000000,0,0,7.9250,C23 C25 C27,1,0,0,0,1
3,1,35.000000,1,0,53.1000,C123,1,0,0,0,1
4,3,35.000000,0,0,8.0500,C23 C25 C27,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
1304,3,29.881138,0,0,8.0500,C23 C25 C27,0,1,0,0,1
1305,1,39.000000,0,0,108.9000,C105,1,0,1,0,0
1306,3,38.500000,0,0,7.2500,C23 C25 C27,0,1,0,0,1
1307,3,29.881138,0,0,8.0500,C23 C25 C27,0,1,0,0,1


In [92]:
data3 = data3[['Pclass', 'Age', 'SibSp','Parch', 'Fare','Sex_female','Sex_male', 'Embarked_C','Embarked_Q','Embarked_S' ]]

In [93]:
data3

,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,3,22.000000,1,0,7.2500,0,1,0,0,1
1,1,38.000000,1,0,71.2833,1,0,1,0,0
2,3,26.000000,0,0,7.9250,1,0,0,0,1
3,1,35.000000,1,0,53.1000,1,0,0,0,1
4,3,35.000000,0,0,8.0500,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...
1304,3,29.881138,0,0,8.0500,0,1,0,0,1
1305,1,39.000000,0,0,108.9000,1,0,1,0,0
1306,3,38.500000,0,0,7.2500,0,1,0,0,1
1307,3,29.881138,0,0,8.0500,0,1,0,0,1


In [94]:
#Scaling¶

#Scaling¶

scaler = StandardScaler()
scaler.fit(data3)

data4 = pd.DataFrame(scaler.transform(data3), index=data3.index, columns=data3.columns)

In [95]:
data4

,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,0.841916,-0.611972,0.481288,-0.445000,-0.503595,-0.743497,0.743497,-0.50977,-0.32204,0.657394
1,-1.546098,0.630431,0.481288,-0.445000,0.734503,1.344995,-1.344995,1.96167,-0.32204,-1.521159
2,0.841916,-0.301371,-0.479087,-0.445000,-0.490544,1.344995,-1.344995,-0.50977,-0.32204,0.657394
3,-1.546098,0.397481,0.481288,-0.445000,0.382925,1.344995,-1.344995,-0.50977,-0.32204,0.657394
4,0.841916,0.397481,-0.479087,-0.445000,-0.488127,-0.743497,0.743497,-0.50977,-0.32204,0.657394
...,...,...,...,...,...,...,...,...,...,...
1304,0.841916,0.000000,-0.479087,-0.445000,-0.488127,-0.743497,0.743497,-0.50977,-0.32204,0.657394
1305,-1.546098,0.708081,-0.479087,-0.445000,1.461829,1.344995,-1.344995,1.96167,-0.32204,-1.521159
1306,0.841916,0.669256,-0.479087,-0.445000,-0.503595,-0.743497,0.743497,-0.50977,-0.32204,0.657394
1307,0.841916,0.000000,-0.479087,-0.445000,-0.488127,-0.743497,0.743497,-0.50977,-0.32204,0.657394


In [96]:
#split the data

In [97]:
#Split Data¶

train_final = data4.loc[:titanic_train.index.max(), :].copy()
test_final = data4.loc[titanic_train.index.max() + 1:, :].reset_index(drop=True).copy()

In [98]:
train_final

,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,0.841916,-0.611972,0.481288,-0.445000,-0.503595,-0.743497,0.743497,-0.50977,-0.322040,0.657394
1,-1.546098,0.630431,0.481288,-0.445000,0.734503,1.344995,-1.344995,1.96167,-0.322040,-1.521159
2,0.841916,-0.301371,-0.479087,-0.445000,-0.490544,1.344995,-1.344995,-0.50977,-0.322040,0.657394
3,-1.546098,0.397481,0.481288,-0.445000,0.382925,1.344995,-1.344995,-0.50977,-0.322040,0.657394
4,0.841916,0.397481,-0.479087,-0.445000,-0.488127,-0.743497,0.743497,-0.50977,-0.322040,0.657394
...,...,...,...,...,...,...,...,...,...,...
886,-0.352091,-0.223721,-0.479087,-0.445000,-0.392417,-0.743497,0.743497,-0.50977,-0.322040,0.657394
887,-1.546098,-0.844922,-0.479087,-0.445000,-0.063719,1.344995,-1.344995,-0.50977,-0.322040,0.657394
888,0.841916,0.000000,0.481288,1.866526,-0.190364,1.344995,-1.344995,-0.50977,-0.322040,0.657394
889,-1.546098,-0.301371,-0.479087,-0.445000,-0.063719,-0.743497,0.743497,1.96167,-0.322040,-1.521159


In [99]:
test_final

,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,0.841916,0.358655,-0.479087,-0.445000,-0.492396,-0.743497,0.743497,-0.50977,3.105202,-1.521159
1,0.841916,1.329283,0.481288,-0.445000,-0.508429,1.344995,-1.344995,-0.50977,-0.322040,0.657394
2,-0.352091,2.494035,-0.479087,-0.445000,-0.456465,-0.743497,0.743497,-0.50977,3.105202,-1.521159
3,0.841916,-0.223721,-0.479087,-0.445000,-0.476284,-0.743497,0.743497,-0.50977,-0.322040,0.657394
4,0.841916,-0.611972,0.481288,0.710763,-0.406194,1.344995,-1.344995,-0.50977,-0.322040,0.657394
...,...,...,...,...,...,...,...,...,...,...
413,0.841916,0.000000,-0.479087,-0.445000,-0.488127,-0.743497,0.743497,-0.50977,-0.322040,0.657394
414,-1.546098,0.708081,-0.479087,-0.445000,1.461829,1.344995,-1.344995,1.96167,-0.322040,-1.521159
415,0.841916,0.669256,-0.479087,-0.445000,-0.503595,-0.743497,0.743497,-0.50977,-0.322040,0.657394
416,0.841916,0.000000,-0.479087,-0.445000,-0.488127,-0.743497,0.743497,-0.50977,-0.322040,0.657394


In [100]:
target

0      0
1      1
2      1
3      1
4      0
      ..
886    0
887    1
888    0
889    1
890    0
Name: Survived, Length: 891, dtype: int64

In [101]:
from pycaret.classification import *

s = setup(train_final, target = target)

,Description,Value
0,Session id,4778
1,Target,Survived
2,Target type,Binary
3,Original data shape,"(891, 11)"
4,Transformed data shape,"(891, 11)"
5,Transformed train set shape,"(623, 11)"
6,Transformed test set shape,"(268, 11)"
7,Numeric features,10
8,Preprocess,True
9,Imputation type,simple


In [102]:
best = compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
gbc,Gradient Boosting Classifier,0.8220,0.8634,0.6819,0.8274,0.7416,0.6088,0.6201,0.1510
catboost,CatBoost Classifier,0.8171,0.8626,0.6609,0.8357,0.7307,0.5962,0.6112,2.0610
lightgbm,Light Gradient Boosting Machine,0.8057,0.8506,0.7152,0.7664,0.7374,0.5838,0.5870,0.1500
lda,Linear Discriminant Analysis,0.8026,0.8586,0.7025,0.7753,0.7312,0.5762,0.5839,0.0310
ridge,Ridge Classifier,0.8010,0.0000,0.6984,0.7744,0.7285,0.5724,0.5803,0.0290
rf,Random Forest Classifier,0.7962,0.8514,0.6902,0.7642,0.7200,0.5609,0.5676,0.3160
et,Extra Trees Classifier,0.7946,0.8294,0.6946,0.7597,0.7193,0.5585,0.5656,0.2660
lr,Logistic Regression,0.7945,0.8586,0.7109,0.7527,0.7253,0.5619,0.5688,0.0350
nb,Naive Bayes,0.7913,0.8330,0.7275,0.7420,0.7275,0.5590,0.5660,0.0280
ada,Ada Boost Classifier,0.7882,0.8487,0.6982,0.7411,0.7134,0.5463,0.5524,0.1500


In [103]:
print(best)

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='log_loss', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_samples_leaf=1,
                           min_samples_split=2, min_weight_fraction_leaf=0.0,
                           n_estimators=100, n_iter_no_change=None,
                           random_state=4778, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)


In [104]:
finalize_model(best)

Pipeline(memory=Memory(location=None),
         steps=[('numerical_imputer',
                 TransformerWrapper(exclude=None,
                                    include=['Pclass', 'Age', 'SibSp', 'Parch',
                                             'Fare', 'Sex_female', 'Sex_male',
                                             'Embarked_C', 'Embarked_Q',
                                             'Embarked_S'],
                                    transformer=SimpleImputer(add_indicator=False,
                                                              copy=True,
                                                              fill_value=None,
                                                              keep_empty_features=False,
                                                              missing_values=nan,
                                                              strategy='mean',
                                                              verbose=...
                                            criterion='friedman_mse', init=None,
                                            learning_rate=0.1, loss='log_loss',
                                            max_depth=3, max_features=None,
                                            max_leaf_nodes=None,
                                            min_impurity_decrease=0.0,
                                            min_samples_leaf=1,
                                            min_samples_split=2,
                                            min_weight_fraction_leaf=0.0,
                                            n_estimators=100,
                                            n_iter_no_change=None,
                                            random_state=4778, subsample=1.0,
                                            tol=0.0001, validation_fraction=0.1,
                                            verbose=0, warm_start=False))],
         verbose=False)

In [105]:
evaluate_model(best)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [106]:
predict_model(best)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Gradient Boosting Classifier,0.8470,0.8767,0.7864,0.8100,0.7980,0.6749,0.6751


,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Survived,prediction_label,prediction_score
502,0.841916,0.000000,-0.479087,-0.445000,-0.496263,1.344995,-1.344995,-0.50977,3.105201,-1.521159,0,1,0.8646
186,0.841916,0.000000,0.481288,-0.445000,-0.344079,1.344995,-1.344995,-0.50977,3.105201,-1.521159,1,1,0.7031
456,-1.546098,2.726986,-0.479087,-0.445000,-0.130425,-0.743497,0.743497,-0.50977,-0.322040,0.657394,0,0,0.8757
410,0.841916,0.000000,-0.479087,-0.445000,-0.491108,-0.743497,0.743497,-0.50977,-0.322040,0.657394,0,0,0.9313
145,-0.352091,-0.844922,0.481288,0.710763,0.066794,-0.743497,0.743497,-0.50977,-0.322040,0.657394,0,0,0.8494
...,...,...,...,...,...,...,...,...,...,...,...,...,...
309,-1.546098,0.009230,-0.479087,-0.445000,0.456963,1.344995,-1.344995,1.96167,-0.322040,-1.521159,1,1,0.9761
193,-0.352091,-2.087325,0.481288,0.710763,-0.141060,-0.743497,0.743497,-0.50977,-0.322040,0.657394,1,1,0.9613
282,0.841916,-1.077873,-0.479087,-0.445000,-0.460091,-0.743497,0.743497,-0.50977,-0.322040,0.657394,0,0,0.9107
20,-0.352091,0.397481,-0.479087,-0.445000,-0.141060,-0.743497,0.743497,-0.50977,-0.322040,0.657394,0,0,0.8085


In [107]:
predictions = predict_model(best, test_final)

In [108]:
predictions

,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,prediction_label,prediction_score
0,0.841916,0.358655,-0.479087,-0.445000,-0.492396,-0.743497,0.743497,-0.50977,3.105201,-1.521159,0,0.8989
1,0.841916,1.329283,0.481288,-0.445000,-0.508429,1.344995,-1.344995,-0.50977,-0.322040,0.657394,1,0.7001
2,-0.352091,2.494035,-0.479087,-0.445000,-0.456465,-0.743497,0.743497,-0.50977,3.105201,-1.521159,0,0.6553
3,0.841916,-0.223721,-0.479087,-0.445000,-0.476284,-0.743497,0.743497,-0.50977,-0.322040,0.657394,0,0.8161
4,0.841916,-0.611972,0.481288,0.710763,-0.406194,1.344995,-1.344995,-0.50977,-0.322040,0.657394,0,0.6464
...,...,...,...,...,...,...,...,...,...,...,...,...
413,0.841916,0.000000,-0.479087,-0.445000,-0.488127,-0.743497,0.743497,-0.50977,-0.322040,0.657394,0,0.9098
414,-1.546098,0.708081,-0.479087,-0.445000,1.461829,1.344995,-1.344995,1.96167,-0.322040,-1.521159,1,0.9454
415,0.841916,0.669256,-0.479087,-0.445000,-0.503595,-0.743497,0.743497,-0.50977,-0.322040,0.657394,0,0.9472
416,0.841916,0.000000,-0.479087,-0.445000,-0.488127,-0.743497,0.743497,-0.50977,-0.322040,0.657394,0,0.9098


In [109]:
test_ids

0       892
1       893
2       894
3       895
4       896
       ... 
413    1305
414    1306
415    1307
416    1308
417    1309
Name: PassengerId, Length: 418, dtype: int64

In [110]:
predictions['prediction_label']

0      0
1      1
2      0
3      0
4      0
      ..
413    0
414    1
415    0
416    0
417    0
Name: prediction_label, Length: 418, dtype: int32

In [111]:
Final_Results = pd.concat([test_ids, predictions], axis =1)

In [112]:
Final_Results

,PassengerId,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,prediction_label,prediction_score
0,892,0.841916,0.358655,-0.479087,-0.445000,-0.492396,-0.743497,0.743497,-0.50977,3.105201,-1.521159,0,0.8989
1,893,0.841916,1.329283,0.481288,-0.445000,-0.508429,1.344995,-1.344995,-0.50977,-0.322040,0.657394,1,0.7001
2,894,-0.352091,2.494035,-0.479087,-0.445000,-0.456465,-0.743497,0.743497,-0.50977,3.105201,-1.521159,0,0.6553
3,895,0.841916,-0.223721,-0.479087,-0.445000,-0.476284,-0.743497,0.743497,-0.50977,-0.322040,0.657394,0,0.8161
4,896,0.841916,-0.611972,0.481288,0.710763,-0.406194,1.344995,-1.344995,-0.50977,-0.322040,0.657394,0,0.6464
...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,0.841916,0.000000,-0.479087,-0.445000,-0.488127,-0.743497,0.743497,-0.50977,-0.322040,0.657394,0,0.9098
414,1306,-1.546098,0.708081,-0.479087,-0.445000,1.461829,1.344995,-1.344995,1.96167,-0.322040,-1.521159,1,0.9454
415,1307,0.841916,0.669256,-0.479087,-0.445000,-0.503595,-0.743497,0.743497,-0.50977,-0.322040,0.657394,0,0.9472
416,1308,0.841916,0.000000,-0.479087,-0.445000,-0.488127,-0.743497,0.743497,-0.50977,-0.322040,0.657394,0,0.9098


In [113]:
submission = Final_Results[['PassengerId', 'prediction_label']].copy()

# Rename the column
submission = submission.rename(columns={'prediction_label': 'survived'})

# Reset the indexes
submission.reset_index(drop=True, inplace=True)

In [114]:
submission

,PassengerId,survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,0
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [116]:
submission.to_csv('C:\\Users\\Auditor\\Desktop\\Victor\\New folder\\Data Science\\Projects\\Final_submissionTitanic2.csv'
, index=False, header=True)